# Installs

In [932]:
#Instalador de las librerias necesarias
#
#%pip install owlready2 agentpy seaborn numpy
#

In [933]:
# Model design
from owlready2 import *
import agentpy as ap
import numpy as np

# Visualization
import seaborn as sns

#Pathfinding
import math
import heapq

#Misc
from matplotlib import pyplot as plt
import IPython
import random

#Assistance Functions (Pathfinding, Grid Sizes, Etc.)

In [934]:
def heuristic(a,b):
  #Distancia de Manhattan, resta el valor absoluto de la XY axtual con la XY destino
  # a[0] = x inicial
  # b[0] = x final
  # a[1] = y inicial
  # b[1] = y final
  return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_neighbors(grid,node):
  neighbors = []

  x,y = node

  if x > 0:
    neighbors.append((x-1,y))

  if x < grid.shape[1] - 1:
    neighbors.append((x+1,y))

  if y > 0:
    neighbors.append((x,y-1))

  if y < grid.shape[0] -1:
    neighbors.append((x,y+1))

  return neighbors

# Ontology

Correr la ontologia **SOLO UNA VEZ** ,si da problemas usar el comando de borrar y volver a crearla

In [935]:
"""
#Creamos la ontologia
onto = get_ontology("file://ontologia.owl")


with onto:
  class Entity(Thing):
    pass

  class Camera(Entity):
    pass

  class Drone(Entity):
    pass

  class Intruder(Entity):
    pass

  class Box(Entity):
    pass

  class Place(Thing):
    pass

  class is_in_place(ObjectProperty):
      domain = [Entity]
      range = [Place]

  class at_position(DataProperty,FunctionalProperty):
        domain = [Place]
        range = [str]

  class is_following_intruder(ObjectProperty):
    domain = [Drone]
    range = [Intruder]

  class is_called_by_camera(ObjectProperty):
    domain = [Drone]
    range = [Camera]
"""

'\n#Creamos la ontologia\nonto = get_ontology("file://ontologia.owl")\n\n\nwith onto:\n  class Entity(Thing):\n    pass\n\n  class Camera(Entity):\n    pass\n\n  class Drone(Entity):\n    pass\n\n  class Intruder(Entity):\n    pass\n\n  class Box(Entity):\n    pass\n\n  class Place(Thing):\n    pass\n\n  class is_in_place(ObjectProperty):\n      domain = [Entity]\n      range = [Place]\n\n  class at_position(DataProperty,FunctionalProperty):\n        domain = [Place]\n        range = [str]\n\n  class is_following_intruder(ObjectProperty):\n    domain = [Drone]\n    range = [Intruder]\n\n  class is_called_by_camera(ObjectProperty):\n    domain = [Drone]\n    range = [Camera]\n'

In [936]:
#Delete just in case
#onto.destroy(update_relation = True, update_is_a = True)

#Camera Agent

In [937]:
class CameraAgent(ap.Agent):

  """
    <-- Funciones de Ejecucion -->
  """

  """
    Funcion de Setup
  """

  def setup(self):
    self.agentType = 0
    self.neighbors = []

    self.actions = {

    }

    self.rules = {

    }


  """
    Funcion de See
  """
  def see(self, e):
    self.neighbors = e.neighbors(self,2)

  """
    Funcion de Next
  """
  def next(self):
    for act in self.actions:
      for rule in self.rules:
        if rule(act):
          act()

  """
    Funcion de Paso
  """

  def step(self):
    self.see(self.model.grid)
    self.next()

    for neighbor in self.neighbors:
      print(self.model.grid.positions[neighbor])

  """
    Funcion de Update
  """

  def update(self):
    pass

  """
    Funcion de End
  """

  def end(self):
    pass

  """
    <-- Acciones -->
  """

  """
    <-- Reglas -->
  """

#Drone Agent

In [938]:
class DroneAgent(ap.Agent):

  """
    <-- Funciones de Ejecucion -->
  """

  """
    Funcion de Setup
  """

  def setup(self):
    self.agentType = 1

    #Variables de patrulla
    #self.path = []
    self.path = [
        # Scaled patrol path to fit a 25x25 grid
        (24, 10), (24, 11), (24, 12), (24, 13), (24, 14), (24, 15), (24, 16), (24, 17), (24, 18), (24, 19), (24, 20), (24, 21), (24, 22), (24, 23),
        (23, 23), (22, 23), (21, 23), (20, 23), (19, 23), (18, 23), (17, 23),
        (17, 22), (17, 21), (17, 20), (17, 19), (17, 18),
        (18, 18), (19, 18), (20, 18), (21, 18), (22, 18), (23, 18), (24, 18),
        (24, 17), (24, 16), (24, 15), (24, 14),
        (23, 14), (22, 14), (21, 14), (20, 14), (19, 14), (18, 14), (17, 14), (16, 14), (15, 14), (14, 14), (13, 14),
        (13, 15), (13, 16), (13, 17), (13, 18), (13, 19), (13, 20), (13, 21), (13, 22), (13, 23),
        (12, 23), (11, 23), (10, 23), (9, 23), (8, 23), (7, 23), (6, 23), (5, 23), (4, 23), (3, 23), (2, 23), (1, 23),
        (1, 22), (1, 21), (1, 20), (1, 19), (1, 18), (1, 17), (1, 16), (1, 15), (1, 14), (1, 13), (1, 12), (1, 11), (1, 10),
        (2, 10), (3, 10), (4, 10), (5, 10), (6, 10), (7, 10), (8, 10), (9, 10), (10, 10), (11, 10), (12, 10), (13, 10),
        (13, 11), (13, 12), (13, 13), (13, 14),
        (15, 14), (16, 14),
        (16, 13), (16, 12), (16, 11), (16, 10), (16, 9), (16, 8), (16, 7), (16, 6), (16, 5), (16, 4),
        (15, 4), (14, 4), (13, 4),
        (13, 5), (13, 6),
        (12, 6), (11, 6), (10, 6), (9, 6), (8, 6), (7, 6), (6, 6), (5, 6), (4, 6), (3, 6), (2, 6), (1, 6),
        (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), 
        (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1),
        (13, 2), (13, 3), (13, 4) ,
        (14, 4), (15, 4), (16, 4), (17, 4), (18, 4),
        (18, 5), (18, 6), (18, 7),
        (19, 7), (20, 7), (21, 7), (22, 7), (23, 7), (24, 7),
        (24, 8), (24, 9)
    ]

    self.path_counter = 0

    """
    grid_length_min = round(self.model.p.M * 0.3 - 1)
    grid_length_max = round(self.model.p.M - (grid_length_min + 1))

    grid_width_min = round(self.model.p.N * 0.3 - 1)
    grid_width_max = round(self.model.p.N - (grid_width_min + 1 ))

    """


    #self.path = [(grid_length_min,grid_width_min), (grid_length_max,grid_length_min), (grid_length_max,grid_width_max),(grid_length_min, grid_width_max) ]

    """
    for i in range(grid_length_min,grid_length_max):
      for j in range(grid_width_min,grid_width_max):
        self.path.append((i,j))
        if reverse:
          self.path_side.reverse()
          reverse = !reverse
        self.path.append(self.path_side)

    for i in range(grid_length_min,grid_length_max):
      self.path.append((i,grid_width_min))

    for i in range(grid_width_min,grid_width_max):
      self.path.append((grid_length_max,i))

    for i in range(grid_length_max,grid_length_min,-1):
      self.path.append((i,grid_width_max))

    for i in range(grid_width_max,grid_width_min,-1):
      self.path.append((grid_length_min,i)) 

      """

    self.actions = (
      self.patrol,
    )

    self.rules = (
      self.patrol_rule,
    )


  """
    Funcion de See
  """
  def see(self, e):
    pass

  """
    Funcion de Next
  """
  def next(self):
    for act in self.actions:
      for rule in self.rules:
        if rule(act):
          act()

  """
    Funcion de Paso
  """

  def step(self):
    self.see(self.model.grid)
    self.next()

  """
    Funcion de Update
  """

  def update(self):
    pass

  """
    Funcion de End
  """

  def end(self):
    pass

  """
    <-- Acciones -->
  """
  def patrol(self):
    print("PATRULLAMOS")
    if(self.path_counter >= len(self.path)):
      self.path_counter = 0
    self.model.grid.move_to(self,self.path[self.path_counter])
    self.path_counter += 1


  """
    <-- Reglas -->
  """
  def patrol_rule(self, act):
    validator = [False]

    if act == self.patrol:
      validator[0] = True

    return sum(validator) == 1

#Guard Agent

In [939]:
class GuardAgent(ap.Agent):

  """
    <-- Funciones de Ejecucion -->
  """

  """
    Funcion de Setup
  """

  def setup(self):
    self.agentType = 2

    self.actions = {

    }

    self.rules = {

    }


  """
    Funcion de See
  """
  def see(self, e):
    pass

  """
    Funcion de Next
  """
  def next(self):
    for act in self.actions:
      for rule in self.rules:
        if rule(act):
          act()

  """
    Funcion de Paso
  """

  def step(self):
    self.see(self.model.grid)
    self.next()

  """
    Funcion de Update
  """

  def update(self):
    pass

  """
    Funcion de End
  """

  def end(self):
    pass

  """
    <-- Acciones -->
  """

  """
    <-- Reglas -->
  """

#DroneModel

In [940]:
class DroneModel(ap.Model):
    def setup(self):
        self.steps = 0

        # Create Agents
        self.cameras = ap.AgentList(self, self.p.cameras, CameraAgent)
        self.drones = ap.AgentList(self, self.p.drones, DroneAgent)
        self.guards = ap.AgentList(self, self.p.guards, GuardAgent)

        # Initialize Grid
        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)
        
        # Define Walls (Obstacle positions)
        self.walls = [
            # Scaled walls to fit a 25x25 grid
            (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14),
            (8, 14), (9, 14), (10, 14), (11, 14), (12, 14),
            (0, 14), (1, 14), (2, 14),
            (8, 14), (7, 14), (6, 14), (5, 14),
            (15, 14), (16, 14), (17, 14), (18, 14), (19, 14), (20, 14), (21, 14), (22, 14), (23, 14), (24, 14), (25, 14), 
            (16, 14), (16, 15), (16, 16), (16, 17), (16, 18), (16, 19), (16, 20), (16, 21), (16, 22), 

            # Guard cabine scaled down
            (0, 20), (1, 20), (3, 20),
            (4, 20), (4, 21), (4, 22), (4, 23), (4, 24)
        ]

        # Place Walls by marking them in the grid (just storing positions)
        self.grid.walls = self.walls  # Store the wall positions in the grid object

        # Hardcoded Positions for Agents
        camera_positions = [(0, 0), (0, 24), (24, 24)]
        guard_position = [(24, 0)]
        drone_position = [(24, 10)]

        # Place Agents
        self.grid.add_agents(self.cameras, camera_positions)
        self.grid.add_agents(self.guards, guard_position)
        self.grid.add_agents(self.drones, drone_position)

    def step(self):
        print(f"Step: {self.steps}")
        self.cameras.step()
        self.drones.step()
        self.guards.step()
        self.steps += 1

#Paremeters

In [941]:
parameters = {
    'M': 25,
    'N': 25,
    'cameras': 3,
    'drones': 1,
    'guards': 1,
    'steps': 200,
}

#Animation

In [942]:
# Custom Plot Function to Show Walls
def animation_plot(model, ax):
    agent_type_grid = model.grid.attr_grid('agentType')
    ax.matshow(agent_type_grid, cmap='Accent', vmin=0, vmax=3)  # Color the grid

    # Draw Walls
    for wall in model.grid.walls:
        ax.add_patch(plt.Rectangle(wall, 1, 1, color='gray'))

    ax.set_title(f"Drone Model - Step {model.t}")

#Simulation

In [943]:
#SIMULATION:

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = DroneModel(parameters)


#Run with animation
#If you want to run it without animation then use instead:
#model.run()
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
IPython.display.HTML(animation.to_jshtml())

Step: 0
PATRULLAMOS
Step: 1
PATRULLAMOS
Step: 2
PATRULLAMOS
Step: 3
PATRULLAMOS
Step: 4
PATRULLAMOS
Step: 5
PATRULLAMOS
Step: 6
PATRULLAMOS
Step: 7
PATRULLAMOS
Step: 8
PATRULLAMOS
Step: 9
PATRULLAMOS
Step: 10
PATRULLAMOS
Step: 11
PATRULLAMOS
Step: 12
PATRULLAMOS
Step: 13
(24, 22)
PATRULLAMOS
Step: 14
(24, 23)
PATRULLAMOS
Step: 15
(23, 23)
PATRULLAMOS
Step: 16
(22, 23)
PATRULLAMOS
Step: 17
PATRULLAMOS
Step: 18
PATRULLAMOS
Step: 19
PATRULLAMOS
Step: 20
PATRULLAMOS
Step: 21
PATRULLAMOS
Step: 22
PATRULLAMOS
Step: 23
PATRULLAMOS
Step: 24
PATRULLAMOS
Step: 25
PATRULLAMOS
Step: 26
PATRULLAMOS
Step: 27
PATRULLAMOS
Step: 28
PATRULLAMOS
Step: 29
PATRULLAMOS
Step: 30
PATRULLAMOS
Step: 31
PATRULLAMOS
Step: 32
PATRULLAMOS
Step: 33
PATRULLAMOS
Step: 34
PATRULLAMOS
Step: 35
PATRULLAMOS
Step: 36
PATRULLAMOS
Step: 37
PATRULLAMOS
Step: 38
PATRULLAMOS
Step: 39
PATRULLAMOS
Step: 40
PATRULLAMOS
Step: 41
PATRULLAMOS
Step: 42
PATRULLAMOS
Step: 43
PATRULLAMOS
Step: 44
PATRULLAMOS
Step: 45
PATRULLAMOS
Step: 46